In [1]:
import sys
sys.path.append(r'./script/')
import copy
from ECMpy_function import *
from cobra.flux_analysis import flux_variability_analysis

# Input and output files

In [2]:
json_model_file="./model/iCW773_irr_enz_constraint_adj_PDH.json"
sbml_path = "./model/iCW773_gpr_uniprot_del.xml"
ecGEM_rxn = pd.read_csv('./data/ecGEM_reactions_list.csv')
GEM_rxn = pd.read_csv('./data/GEM_reactions_list.csv')

obj='CG_biomass_cgl_ATCC13032'# CG_biomass_cgl_ATCC13032 EX_lys_L_e EX_trp_L_e
substrate='EX_glc_e'

fva_ec773max_outfile='./analysis/fva_ec773max.csv'
fva_773max_outfile='./analysis/fva_773max.csv'
fva_max_diff_outfile='./analysis/fva_max_diff.csv'

fva_distrbutionmax_fig='./analysis/fva_distrbutionmax.png'

# μmax

In [3]:
# substate concentration
#ecGEM
enz_model=get_enzyme_constraint_model(json_model_file)
enz_model.reactions.get_by_id(substrate).bounds=(-1000,0)
enz_model.reactions.get_by_id(substrate+'_reverse').bounds=(0,0)
enz_model.objective=obj
enz_model_solution = cobra.flux_analysis.pfba(enz_model)
print("ecGEM glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
print("ecGEM growth rates:"+str(enz_model_solution.fluxes[obj]))

0.12
ecGEM glucose uptake rates:6.280024540326834
ecGEM growth rates:0.4848207420373486


In [4]:
#FVA ecGEM
enz_model=get_enzyme_constraint_model(json_model_file)
enz_model.objective=obj
enz_model.reactions.get_by_id(substrate).bounds=(-6.3,-6.3)
enz_model.reactions.get_by_id(substrate+'_reverse').bounds=(0,0)
enz_model.reactions.get_by_id(obj).bounds=(0,enz_model_solution.fluxes[obj])
enz_model.objective=obj
enz_model_solution = cobra.flux_analysis.pfba(enz_model)
print("ecGEM glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
print("ecGEM growth rates:"+str(enz_model_solution.fluxes[obj]))
#FVA GEM
cg_model=cobra.io.read_sbml_model(sbml_path)
cg_model.objective=obj
cg_model.reactions.get_by_id(substrate).bounds=(enz_model_solution.fluxes[substrate],enz_model_solution.fluxes[substrate])
cg_model.reactions.get_by_id(obj).bounds=(0,enz_model_solution.fluxes[obj])
cg_model.objective=obj
cg_model_solution = cobra.flux_analysis.pfba(cg_model)
print("GEM glucose uptake rates:"+str(-cg_model_solution.fluxes[substrate]))
print("GEM growth rates:"+str(cg_model_solution.fluxes[obj]))

0.12
ecGEM glucose uptake rates:6.3
ecGEM growth rates:0.4847953057880492
GEM glucose uptake rates:6.3
GEM growth rates:0.4847953057880492


## ecGEM

In [5]:
ecGEM_rxn.set_index('id', inplace=True)
ecFVA = flux_variability_analysis(enz_model, enz_model.reactions, loopless=False)
ecFVA.append(ecGEM_rxn)
ecFVA['ec773differernce'] = ecFVA['maximum'] - ecFVA['minimum']
ecFVA.reset_index()
ecFVA_name = pd.concat([ecGEM_rxn,ecFVA],axis = 1)
ecFVA_name = ecFVA_name.iloc[:, 4:7]

#isozyme
rex_name = []
for row in ecFVA_name.itertuples():
    if re.search('num', str(row.Index)):
        rex_name.append(str(row.Index.split('_num')[0]))
    else:
        rex_name.append(str(row.Index))
ecFVA_name['rex_name'] = rex_name

#FV max
FV_max = ecFVA_name.sort_values('ec773differernce', ascending=False).drop_duplicates(subset=['rex_name'], keep='first')
FV_max['id'] = FV_max.index
FV_max1 = FV_max.set_index('rex_name', drop=False)

#combine reverse reaction FV
reverse_list = []
rex_list = []
for row in FV_max1.itertuples():
    rex_list.append(str(row.Index))
    if re.search('reverse', str(row.Index)):
        reverse_list.append(str(row.Index).split('_reverse')[0])
FV_max_reverse = pd.DataFrame()
for row in FV_max1.itertuples():
    if str(row.Index) and str(row.Index)+'_reverse' in rex_list:
        FV_max_reverse.loc[row.Index,'id'] = row.Index
        FV_max_reverse.loc[row.Index,'ec773differernce'] = FV_max1.loc[row.Index, 'ec773differernce'] - FV_max1.loc[str(row.Index) + '_reverse', 'ec773differernce']
    elif re.search('reverse', str(row.Index)):
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0],'id'] = str(row.Index).split('_reverse')[0]
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0].split('_reverse')[0],'ec773differernce'] = FV_max1.loc[str(row.Index).split('_reverse')[0], 'ec773differernce'] - FV_max1.loc[row.Index, 'ec773differernce']

    else:
        FV_max_reverse.loc[row.Index,'id'] = row.Index
        FV_max_reverse.loc[row.Index,'ec773differernce']  = FV_max1.loc[row.Index, 'ec773differernce']    
ecFV_result = FV_max_reverse.set_index('id', inplace=False)
ecFV_result.to_csv(fva_ec773max_outfile)


## GEM

In [6]:
GEM_rxn.set_index('id', inplace=True)
GEM_FVA = flux_variability_analysis(cg_model, cg_model.reactions, loopless=False)
GEM_FVA.append(GEM_rxn)
GEM_FVA['773differernce'] = GEM_FVA['maximum'] - GEM_FVA['minimum']
GEM_FVA.to_csv(fva_773max_outfile)
# combine results
FVA_total = pd.concat([GEM_FVA,ecFV_result],axis = 1)
FVA_total.to_csv(fva_max_diff_outfile)

In [7]:
import numpy as np
import plotly.graph_objects as go

df_fv = pd.read_csv(fva_max_diff_outfile)
df_fv


col1 = 'ec773differernce'
col2 = '773differernce'
df_ec = df_fv[col1]
df_ec = abs(df_ec)
df_ec.replace(0, np.nan, inplace=True)
df_ec = df_ec.dropna(axis=0)

df_GEM = df_fv[col2]
df_GEM = abs(df_GEM)
df_GEM.replace(0, np.nan, inplace=True)
df_GEM = df_GEM.dropna(axis=0)

df_ec.sort_values(axis=0, inplace=True)
df_ec = df_ec.reset_index(drop=True)
df_GEM.sort_values(axis=0, inplace=True)
df_GEM = df_GEM.reset_index(drop=True)

y_index1 = df_ec.index / (len(df_ec) - 1)
y_index2 = df_GEM.index / (len(df_GEM) - 1)

data1_cdf_data = df_ec  # cdf 
data2_cdf_data = df_GEM

trace0 = go.Scatter(x=data1_cdf_data,
                    y=y_index1,
                    mode='lines',
                    name='ec<i>i</i>CW773',
                    line={'color': 'red', 'width': 4})
                    # marker={'color': 'red', 'width':0.5})
trace1 = go.Scatter(x=data2_cdf_data,
                    y=y_index2,
                    line={'color': 'blue', 'width': 4},
                    mode='lines',
                    name='<i>i</i>CW773',
                    xaxis='x2',
                    yaxis="y2"
                    )
data1 = [trace0,trace1]
layout = go.Layout(
    plot_bgcolor='white',
    xaxis=dict(title=dict(text="<b>Flux variability(mmol/gDW/h)(μ<sub>max</sub>)</b>",
                          font=dict(size=20, family='Times New Roman')),
               type="log",
               rangemode="tozero",
               range=[-12.1, 3],
               tickfont=dict(color='black', size=15, family='Times New Roman'),
               linecolor='black',
               ticks='inside',
               tickcolor='black',
               showexponent = 'all', 
               exponentformat =  "power",
               zeroline=False,
#                gridcolor="yellow"
              ),
    xaxis2=dict(
                linecolor='black',
                tickcolor='black',
                showticklabels=False,
                range=[-12.1, 3],
                type="log",
                tickfont=dict(color='black', size=15, family='Times New Roman'),
                rangemode="tozero",
                overlaying='x',
                side='top',
                nticks=5500,
                zeroline=False,
                showexponent = 'all', 
                exponentformat =  "power",
                gridcolor="white"
                #                              dtick = 0.05,
                ),
    yaxis=dict(title=dict(text="<b>Cummulative distribution</b>",
                          font=dict(size=20, family='Times New Roman')),
               range=[0, 1],
               showgrid=False,
               zeroline=False,
               rangemode="tozero",
               tickfont=dict(color='black', size=15, family='Times New Roman'),
               ticks='inside',
               tickcolor='black',
               linecolor='black'),
    yaxis2=dict(
                range=[0, 1],
                linecolor='black',
                tickcolor='black',
                # ticks='inside',
                showgrid=False,
                zeroline=False,
                rangemode="tozero",
                tickfont=dict(color='black', size=15, family='Times New Roman'),
                showticklabels=False,
                overlaying='y',
                side='right'
                ),
    legend=dict(x=0.05, y=0.95, 
                font=dict(size=15, color="black",family='Times New Roman')),
    height=450,
    width=800,
    margin=go.layout.Margin(l=0, r=20, b=0, t=10))

fig = go.Figure(data1, layout=layout)
fig.add_hline(y=0.5,line_width=3,line_color="orange")
fig.write_image(fva_distrbutionmax_fig)
fig.show()